## MOwNiT - laboratorium 7
### Iteracyjne rozwiązywanie układów równań liniowych
https://github.com/piotrMocz/mownit2/blob/master/Lab4.ipynb

### Zadania:
- zadanie wstępne: zaimplementować metodę Newtona znajdywania zer funkcji rzeczywistych
- zaimplementować dowolną metodę iteracyjną rozwiązywania układów równań liniowych
- przetestować powyższą metodę dla wygenerowanych macierzy (2x2, 3x3, 4x4) i sprawdzić jej poprawność
- sprawdzić, jak liczba iteracji wpływa na dokładność wyniku (wykres jest bardzo dogodną reprezentacją tego wyniku)
- (*) porównać zbieżność metod: Jacobiego, Gaussa-Seidla, SOR i Conjugate Gradient

In [12]:
import numpy as np

def newton_raphson(a, b, fun):
    x = a
    if (fun(x) * fun(b) >= 0):
        raise ValueError("f(a) * f(b) must be < 0")
    while (fun(x) != 0):
        h = 0.001
        d = (fun(x + h) - fun(x)) / h
        x = x - fun(x) / d
    return x

def f(x): 
    return x**3 + 2

print(newton_raphson(-5, -1, f))

-1.2599210498948732


In [13]:
def check_method(A: np.matrix, b: np.matrix, method):
    xt = np.linalg.solve(A, b)
    for i in range(1,25):
        x = method(A, b, i)
        print(x)
        if(np.allclose(xt.transpose(), x) == True):
            print("Accurate after", i, "iterations")
            break

#### Oznaczenia do wzorów
$A = D+L+U$, gdzie: $D$ - diagonala ($a_{ii}$), $L$ - pod diagonalą ($a_{ij}, j<i$), $U$ - nad diagonalą ($a_{ij}, j>i$)

$\begin{cases} M = I - B^{-1}A \\ W = B^{-1}b  \end{cases} \Longrightarrow x^{(k+1)} = Mx^{(k)} + W$

#### Metoda Jacobiego
$B = D$

$M = I - D^{-1}(D+L+U) = -D^{-1}(L+U)$

Wzór roboczy:

$x^{(k+1)}_{i} = \frac{1}{a_{ii}}\Big[b_{i} - \sum\limits_{j=1,j \neq i}^{n} a_{ij}x_{j}^{(k)}\Big]$

Wzór macierzowy:

$x^{(k+1)} = -D^{-1}(L+U)x^{(k)} + D^{-1}b$

In [14]:
def iterative_jacobi(A: np.matrix, b: np.matrix, iterations: int=1000) -> np.matrix:    
    n = A.shape[0]
    x = np.zeros(n)
    k = 0
    while k < iterations:
        x1 = np.zeros(n)
        for i in range(n):
            sum = 0
            for j in range(n):
                if j != i:
                    sum = sum + A[i, j] * x[j]
            x1[i] = (1 / A[i, i]) * (b[i] - sum)
        x = x1
        k = k + 1
    return x

In [15]:
A = np.matrix([[2, 1], [5, 7]])
b = np.matrix([11, 13]).transpose()
check_method(A, b, iterative_jacobi)

[5.5        1.85714286]
[ 4.57142857 -2.07142857]
[ 6.53571429 -1.40816327]
[ 6.20408163 -2.81122449]
[ 6.90561224 -2.57434402]
[ 6.78717201 -3.07543732]
[ 7.03771866 -2.99083715]
[ 6.99541858 -3.16979904]
[ 7.08489952 -3.1395847 ]
[ 7.06979235 -3.20349966]
[ 7.10174983 -3.19270882]
[ 7.09635441 -3.21553559]
[ 7.1077678  -3.21168172]
[ 7.10584086 -3.21983414]
[ 7.10991707 -3.21845776]
[ 7.10922888 -3.22136934]
[ 7.11068467 -3.22087777]
[ 7.11043889 -3.22191762]
[ 7.11095881 -3.22174206]
[ 7.11087103 -3.22211344]
[ 7.11105672 -3.22205074]
[ 7.11102537 -3.22218337]
[ 7.11109168 -3.22216098]
[ 7.11108049 -3.22220835]
Accurate after 24 iterations


In [16]:
A = np.array([[10.0, -1.0, 2.0, 0.0],
              [-1.0, 11.0, -1.0, 3.0],
              [2.0, -1.0, 10.0, -1.0],
              [0.0, 3.0, -1.0, 8.0]])
b = np.array([6.0, 25.0, -11.0, 15.0]).transpose()
check_method(A, b, iterative_jacobi)

[ 0.6         2.27272727 -1.1         1.875     ]
[ 1.04727273  1.71590909 -0.80522727  0.88522727]
[ 0.93263636  2.05330579 -1.04934091  1.13088068]
[ 1.01519876  1.95369576 -0.96810863  0.97384272]
[ 0.9889913   2.01141473 -1.0102859   1.02135051]
[ 1.00319865  1.99224126 -0.99452174  0.99443374]
[ 0.99812847  2.00230688 -1.00197223  1.00359431]
[ 1.00062513  1.9986703  -0.99903558  0.99888839]
[ 0.99967415  2.00044767 -1.00036916  1.00061919]
[ 1.0001186   1.99976795 -0.99982814  0.99978598]
[ 0.99994242  2.00008477 -1.00006833  1.0001085 ]
[ 1.00002214  1.99995896 -0.99996916  0.99995967]
[ 0.99998973  2.00001582 -1.00001257  1.00001924]
[ 1.00000409  1.99999268 -0.99999444  0.9999925 ]
Accurate after 14 iterations


#### Metoda S-R (Gaussa-Seidla)
$B = D + L$

$M = I - B^{-1}(B+U) = -B^{-1}U$

$x^{(k+1)} = -B^{-1}Ux^{(k)} + B^{-1}b$

$(D+L)x^{(k+1)} = - Ux^{(k)} + b$

Wzór roboczy:

$x^{(k+1)}_{i} = \frac{1}{a_{ii}}\Big[b_{i} - \sum\limits_{j=1}^{i-1} a_{ij}x_{j}^{(k+1)} - \sum\limits_{j=i+1}^{n} a_{ij}x_{j}^{(k)}\Big]$

Wzór macierzowy:

$x^{(k+1)} = -D^{-1}Lx^{(k+1)} -D^{-1}Ux^{(k)} + D^{-1}b$

In [17]:
def iterative_gs(A: np.matrix, b: np.matrix, iterations: int=1000) -> np.matrix:
    n = A.shape[0]
    x = np.zeros(n)
    k = 0
    while k < iterations:
        x1 = np.zeros(n)
        for i in range(n):
            sumL = 0
            sumU = 0
            for j in range(0, i):
                sumL = sumL + A[i,j] * x1[j]
            for j in range(i+1, n):
                sumU = sumU + A[i,j] * x[j]
            x1[i] = (1 / A[i,i]) * (b[i] - sumL - sumU)
        x = x1
        k = k + 1
    return x

In [18]:
A = np.matrix([[2, 1], [5, 7]])
b = np.matrix([11, 13]).transpose()
check_method(A, b, iterative_gs)

[ 5.5        -2.07142857]
[ 6.53571429 -2.81122449]
[ 6.90561224 -3.07543732]
[ 7.03771866 -3.16979904]
[ 7.08489952 -3.20349966]
[ 7.10174983 -3.21553559]
[ 7.1077678  -3.21983414]
[ 7.10991707 -3.22136934]
[ 7.11068467 -3.22191762]
[ 7.11095881 -3.22211344]
[ 7.11105672 -3.22218337]
[ 7.11109168 -3.22220835]
Accurate after 12 iterations


In [19]:
A = np.array([[10.0, -1.0, 2.0, 0.0],
              [-1.0, 11.0, -1.0, 3.0],
              [2.0, -1.0, 10.0, -1.0],
              [0.0, 3.0, -1.0, 8.0]])
b = np.array([6.0, 25.0, -11.0, 15.0]).transpose()

check_method(A, b, iterative_gs)

[ 0.6         2.32727273 -0.98727273  0.87886364]
[ 1.03018182  2.03693802 -1.0144562   0.98434122]
[ 1.00658504  2.00355502 -1.00252738  0.99835095]
[ 1.00086098  2.00029825 -1.00030728  0.99984975]
[ 1.00009128  2.00002134 -1.00003115  0.9999881 ]
[ 1.00000836  2.00000117 -1.00000275  0.99999922]
Accurate after 6 iterations


#### Metoda SOR
$\Rightarrow$ modyfikacja S-R $\Rightarrow$ $r_{i}^{(k)}$ to poprawka do starego rozwiązania $x_{i}^{(k)}$.

Następuje przyśpieszenie zbieżności: $x_{i}^{(k+1)} = x_{i}^{(k)} + \omega r_{i}^{(k)}$

Postać wzoru:

$x^{(k+1)}_{i} = (1-\omega)x_{i}^{(k)} + \frac{\omega}{a_{ii}}\Big[b_{i} - \sum\limits_{j=1}^{i-1} a_{ij}x_{j}^{(k+1)} - \sum\limits_{j=i+1}^{n} a_{ij}x_{j}^{(k)}\Big]$

Postać macierzowa:

$x^{(k+1)} = (1-\omega)x^{(k)} + \omega D^{-1}\Big[b - Lx^{(k+1)} - Ux^{(k)}\Big]$

In [20]:
def iterative_sor(A: np.matrix, b: np.matrix, iterations: int=1000) -> np.matrix:
    n = A.shape[0]
    x = np.zeros(n)
    omega = 1.44
    k = 0
    while k < iterations:
        x1 = np.zeros(n)
        for i in range(n):
            sumL = 0
            sumU = 0
            for j in range(0, i):
                sumL = sumL + A[i,j] * x1[j]
            for j in range(i+1, n):
                sumU = sumU + A[i,j] * x[j]
            x1[i] = (1 - omega) * x[i] + (omega / A[i,i]) * (b[i] - sumL - sumU)
        x = x1
        k = k + 1
    return x

In [21]:
A = np.matrix([[2, 1], [5, 7]])
b = np.matrix([11, 13]).transpose()
check_method(A, b, iterative_sor)

[ 7.92  -5.472]
[ 8.37504    -3.53236114]
[ 6.77828242 -2.74342302]
[ 6.91282031 -3.22893762]
[ 7.20319415 -3.31398143]
[ 7.1366612  -3.20812827]
[ 7.08972142 -3.20642274]
[ 7.10914695 -3.22715371]
[ 7.11552602 -3.22459341]
[ 7.11087581 -3.22093687]
[ 7.11028919 -3.22194237]
[ 7.11127126 -3.22251008]
[ 7.1112479  -3.22223626]
[ 7.11106103 -3.22216453]
[ 7.11109161 -3.22222755]
Accurate after 15 iterations


In [22]:
A = np.array([[10.0, -1.0, 2.0, 0.0],
              [-1.0, 11.0, -1.0, 3.0],
              [2.0, -1.0, 10.0, -1.0],
              [0.0, 3.0, -1.0, 8.0]])
b = np.array([6.0, 25.0, -11.0, 15.0]).transpose()

check_method(A, b, iterative_sor)

[ 0.864       3.38583273 -1.34527209  0.62950135]
[ 1.35883827  1.53751446 -1.07137543  1.39991402]
[ 0.79606936  2.01039643 -0.85077808  0.8452837 ]
[ 1.04825065  2.08203783 -1.09001953  1.00757122]
[ 1.01650879  1.9513067  -0.97106752  1.02817089]
[ 0.97739174  2.01118947 -1.00055122  0.98146327]
[ 1.01171767  2.0038183  -1.0052516   1.00514899]
[ 0.99690652  1.99520535 -0.99674735  1.00090903]
[ 0.99973394  2.00214362 -1.00091496  0.99827778]
[ 1.00068926  1.99970363 -1.0000866   1.00090223]
[ 0.99967899  1.99972271 -0.99977945  0.99979245]
[ 1.0000378   2.00023734 -1.00010364  0.99994451]
[ 1.00004739  1.99991    -0.999989    1.000075  ]
[ 0.99996302  2.00000674 -0.99998242  0.99996652]
[ 1.00001218  2.00001408 -1.00001404  1.0000046 ]
[ 1.00000071  1.99999026 -0.99999477  1.00000418]
Accurate after 16 iterations
